# Fractional Occupations as Thermal Occupations.
 A mixing of my work with Psi4's fractional occupations and VM's work on the thermal Occupations

### Import and Set up Section
Memory is likely set way too large but that is safest.

In [1]:
%matplotlib inline

In [2]:
import psi4
import matplotlib.pyplot as plt
import numpy as np

psi4.set_options({"save_jk" : True})
psi4.set_memory(int(15.50e9))
psi4.set_num_threads(10)
psi4.core.clean()

import n2v

import matplotlib as mpl
mpl.rcParams["font.size"] = 11
mpl.rcParams["font.family"] = "sans-serif"
mpl.rcParams["axes.edgecolor"] = "#eae8e9" 
import scipy.sparse as spa
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from scipy.integrate import simpson
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib import cm
from decimal import Decimal
from collections import Counter


  Memory set to  14.435 GiB by Python driver.
  Threads set to 10 by Python driver.


#### Molecular Set-Up
##### Define Multiple Molecules to work with

In [15]:
# Molecules should all work
F = psi4.geometry(
    """
    0 2
    F 0.0 0.0 0.0
    noreorient
    nocom
    units bohr
    symmetry c1
    """, name="F")
H2O = psi4.geometry(
    """
    0 1
    O       -0.9937662132      1.1143410089     -0.0299235442                 
    H        0.8352748626      1.0608684271      0.0396432808                 
    H       -1.5207147848      0.0456065199      1.3604799627                   
    symmetry c1
    """, name="Water")
CH4 = psi4.geometry(
    """ 
    C       -0.70207        1.95893        0.00000
    H        0.39013        1.95893       -0.00000
    H       -1.06613        1.99095        1.02924
    H       -1.06613        1.05158       -0.48689
    H       -1.06614        2.83427       -0.54235
    noreorient
    nocom
    units bohr
    symmetry c1
    """, name="Methane")
CCCC = psi4.geometry(
    """ 
    C        0.60024       -1.20973        0.11111
    C       -0.36574        2.49231        0.54931
    C       -1.34777       -0.08504        3.23689
    C        2.34127        0.92942        2.79869
    noreorient
    nocom
    units bohr
    symmetry c1
    """, name="Graphite")
Ne = psi4.geometry(
    """
    0 1
    Ne 0.0 0.0 0.0
    noreorient
    nocom
    units bohr
    symmetry c1
    """, name="Ne")
NH3 =psi4.geometry(
    """
    0 1
    N       -0.58187        0.48713       -0.03712
    H        0.43546        0.50133        0.01948
    H       -0.89347        1.40498        0.27725
    H       -0.89347       -0.17373        0.67317
    noreorient
    nocom
    units bohr
    symmetry c1
    """, name="Ammonia")


## Functions used in the calculation

### VM's Secant method code

In [4]:
def secant(x0,x1,fx0,fx1):
    '''
    the update process for the secant method
    '''
    return (x0*fx1-fx0*x1)/(fx1-fx0)

def secant_method(x0,x1,func,criterion=1e-6,max_iter=100):
    '''
    Description: Based on the secant method page on wikipedia
    takes the first two guesses at the correct root and a defined
    function then run the secant method to find the root.
    INPUT:
        x0: Scalar
            First guess at root value
        x1: scalar
            Second guess at root value
        func: Scalar
            Function to find root of
    OUTPUT
        x1: Scalar
            Root value
        fx1: Scalar
            function evaluated at root value            
    '''
    
    i = 0
    
    fx0 = func(x0)
    fx1 = func(x1)
    conv = abs(min([fx0,fx1]))
    
    while conv > criterion:
        xt = secant(x0,x1,fx0,fx1)
        
        fx0 = fx1
        x0 = x1
        x1 = xt
        fx1 = func(x1)
        
        conv = abs(fx1)
        
        if i > max_iter:
            break
        i += 1
        
    return x1,fx1

### Psi4 Options function

In [5]:
def Psi4Opts(testing=bool, Fractional=bool, frac_occ=None, frac_val=None):
    '''
    Description: Sets the various options needed for the Psi4 calcualtions.
    Current Capabilities: Does all that it is desired
    Goal: NA
    Note: Pylance does not like the fact that I am using booleans as imputs. Pretty sure that is just a pylance thing
    INPUT:
        Use: Bool 
            True: Sets options Always needs to be true to use Psi4
        testing: Bool
            True: uses a predetermined set of values for the frac_occ and frac_val
            False: Allows for either no fractional occupations to be used or for user specfied values to be used
        Working: Bool
            True: Allows for user specfied values to be used for frac_occ and frac_val
            False: Allows for either no fractional occupations to be used or for examples values to be used
        frac_occ: ndarray
            The absolute orbital indicies to be fractionally occupied
        frac_val: ndarray
            The ammount to fractionally occupy the specified orbitals
    OUTPUT
        No output simply calls a Psi4 option command and sets options to be used
    '''
    options = {
        'basis': 'sto-6g',
        'scf_type': 'direct',
        'reference': 'uhf',
        'df_scf_guess': False,
        'e_convergence': 8,
        "opdm": True,
        'tpdm': True,
        'DFT_SPHERICAL_POINTS': 74,
        'DFT_RADIAL_POINTS': 56,
        'MAXITER' : 1500,
    }

    if Fractional and testing:
        options.update({
            'frac_start': 1,
            'frac_occ' : [ +3, +4, +5, -3, -4, -5 ],
            'frac_val' : [1.0, 1.0, 1.0, 6.666666666666666e-01, 6.666666666666666e-01, 6.666666666666666e-01],
        })
    if Fractional:
        options.update({
            'frac_start': 1,
            'frac_occ': frac_occ,
            'frac_val': frac_val,
        })

    psi4.set_options(options)


### Temperature converter
Unsure if this will be needed at all but it is better to have it and not need than need it and not have it.

In [6]:
## Converting real temperature to electronic temperature
def convert_temp(temp, arrays="", reverse=""):
    '''
    Description: Calculates the Fermi-weighting to be used in the frac_val array
    Current Capabilities: As far as I know this works just fine
    Goal: NA
    Note: Temperature is converted to electronic temperature. Chemical potential will be wrong until its function is corrected.
            
    INPUT:
        temp: Scalar
            Temperature in [K] 
    OUTPUT
        tau: scalar
            The electronic temperature 
    '''
    K_b = 3.166811563e-6
    if reverse == "No":
        if arrays == "No":
            tau = K_b * temp
            print("For a real temperature of", temp, "K the electronic temperature should be: ", tau)
            return tau
        if arrays == "Yes":
            taus = np.empty(len(temp))
            for i, t in enumerate(temp):
                taus[i] = K_b * temp[i]
            return taus
    if reverse == "Yes":
        if arrays == "No":
            tau = temp/K_b
            print("For an electronic temperature of {tau}  the real temperature should be: {temp}")
            return tau
        if arrays == "Yes":
            taus = np.empty(len(temp))
            for i,t in enumerate(temp):
                taus[i] = temp[i]/K_b 
        return taus

#temp = 298.15
#reverse_test = 0.00094418486750845
#tau = convert_temp(temp, arrays="No")
#T = convert_temp(reverse_test, arrays="No", reverse="Yes")
#temps = np.linspace(1,100,1001)
#print(temps)
#convert_temp(temps, arrays="Yes", reverse="Yes")
#print(tau)
#print("For a real temperature of", temp, "K the electronic temperature should be: ", convert_temp(temp))

### Fermi Occupation Function
Currently accepts non-electronic temperatures. Should be updated to take electronic temperatures. I have it set up this way in case something I need for Psi4 needs the non-electronic temperature

In [7]:
def fermi_weighting(energy, chemical_potential, temperature):
    '''
    Description: Calculates the Fermi-weighting to be used in the frac_val array
    Current Capabilities: As far as I know this works just fine
    Goal: NA
    Note: Temperature is converted to electronic temperature. Chemical potential will be wrong until its function is corrected.
            
    INPUT:
        energy: Scalar
            Energy of the system
        temperature: Scalar
            Temperature in [K] Needs to be converted to electronic temperature using convert_temp
        chemical_potential: Scalar
            Chemical potential of the sytem (temperature dependent)
    OUTPUT
        mu: scalar
            The fermi weight at the given energy level, chemicial potential and temperature
    '''
    tau = temperature
    Eis = energy
    mu = chemical_potential
    fermi = 1/(1+np.exp((Eis-mu)/tau))
    return fermi

### Density Function
Similar to VM's but this one has options for Non-Fermi_weighted densities, and Fermi-Weighted densities

In [8]:
def dens_function(vecs, x, fs, FWs="None"):
    '''
    Description: Calculates the density using the eigenvectors of 
                 the system
    INPUT:
        fs: ndarray
            The Fermi weights 
        vecs: ndarray
            The eigenvectors of the system Hopefully from Psi4
        x: ndarray
            Linspace
        FWs: String
            Switch for generating either weighted or non weighted 
            densities
    OUTPUT
        Dens: ndarray
            The density for the system
    '''

    dens = np.zeros(len(x))
    if FWs == "yes":
        for i,f in enumerate(fs):
            dens+=2*f*vecs[:,i]**2
    if FWs == "no":
        for i  in range(len(vecs[:,1])):
            dens = 2 * vecs[:,i]**2
    return dens

### Particle Number Function

In [9]:
def particle_number_function(mu,tau,vals,vecs,x, FWs="None"):
    '''
    Description: Determine the Unshifted particle number
    INPUT:
        mu: Scalar (float)
            chemical potential
        tau: Scalar (float)
            Electronic temperature
        vecs: ndarray
            Eigenvectors Hopefully returned from Psi4
        vals: ndarray
            Eigenvalues Easily returned from Psi4
        x: ndarray
            Grid
    OUTPUT
        Ne: Scalar (float)
            The unshifted particle number
    '''
    
    dens = np.zeros(len(x))
    if FWs == "yes":
        fs = fermi_weighting(vals,mu,tau)
        for i,f in enumerate(fs):
            dens+=2*f*vecs[:,i]**2
    if FWs == "no":
        for i  in range(len(vecs[:,1])):
            dens = 2 * vecs[:,i]**2
    Ne = np.trapz(dens,x)
    return Ne

### Particle Number Shifter

In [10]:
def particle_number_shifter(tau,vals,vecs,x,target_Ne): 
    
    '''
    Old title: particle_number_function_function

    Description: Determines the Shifted particle number
    INPUT:
        tau: Scalar (float)
            Electronic temperature
        vecs: ndarray
            Eigenvectors
        vals: ndarray
            Eigenvalues 
        x: ndarray
            Grid
        target_Ne: Scalar (int)
            The desired particle number
    OUTPUT
        particle_number_Shift: Scalar (float)
            The Shifted particle number
    '''

    def particle_number_Shift(mu):
        
        fs = fermi_weighting(vals,mu,tau)
        dens = np.zeros(len(x))
        for i,f in enumerate(fs):
            dens += 2*f*vecs[:,i]**2
        Ne = np.trapz(dens,x)
        
        return Ne - target_Ne

    
    return particle_number_Shift

## The Calculations

In [11]:
# Setting up the calculation
taus = np.linspace(1,100,1001)
#Physical_Temps = convert_temp(taus, arrays="Yes", reverse="Yes")
Nx = 58
nk = 58
x = np.linspace(-50,50,Nx)
dtau = taus[1]-taus[0]
mus = np.empty(len(taus))
Nes = np.empty(len(taus))
mu0s = np.empty(len(taus))
mu1s= np.empty(len(taus))
mu_upper = np.empty(len(taus))
mu0 = 1
mu1 = 5


In [28]:
nitrotest = psi4.geometry("""
symmetry c1
0 1
C           -0.095064772343     0.146295623041     0.059537205186
C            1.283018363291     0.142649668478     0.196784140588
C            1.990331050963    -0.960422939516    -0.254006621934
C            1.318416263743    -2.031528686933    -0.828747057589
C           -0.064188925162    -2.007366882283    -0.956737400211
C           -0.784558755686    -0.910752841459    -0.510319723340
N           -0.848855091435     1.308105436534     0.533445635977
O           -0.233820735922     2.201021978360     1.018562061794
O           -2.029554627386     1.286506572614     0.404620639986
H            1.779858487179     0.986578029758     0.646345969034
H            3.066459468369    -0.982350238052    -0.155873129710
H            1.875676025875    -2.889960105077    -1.178879784359
H           -0.584173157007    -2.842448011438    -1.404447615844
H           -1.857675444135    -0.866918749314    -0.597782154057
""", name = "nitrobenzene")
H2O = psi4.geometry(
    """
    0 1
    O       -0.9937662132      1.1143410089     -0.0299235442                 
    H        0.8352748626      1.0608684271      0.0396432808                 
    H       -1.5207147848      0.0456065199      1.3604799627                   
    symmetry c1
    nocom
    units bohr
    noreorient
    """, name="Water")
psi4.set_output_file(F'{name}_geometry_initial.dat', False)
H2O.print_out_in_angstrom()
H2O.print_distances()

In [29]:
# Calculating
Psi4Opts(False, False)  # type: ignore
Molec, name = nitrotest, "nitrobenzene"
Water_Energy, Water_wfn = psi4.energy("svwn/cc-pvtz", molecule = Molec, return_wfn = True, write_orbitals=('test.dat', True)) # type: ignore
Water_frequency, Water_wfn2 = psi4.frequency("svwn/cc-pvtz", molecule = Molec, return_wfn = True)
## Small inline clean-up to remove those pesky psi.clean files 
%rm psi*.clean
print("The total energy of the specified", name, "molecule is:", Water_Energy)

Following warning is harmless if you've altered chgmult through `set_molecular_change` or `set_multiplicity`. Such alterations are an expert feature. Specifying in the original molecule string is preferred. Nonphysical masses may also trigger the warning.


ValidationError: Following atoms are too close: [(0, np.int64(1), np.float64(0.0)), (0, np.int64(2), np.float64(0.0)), (0, np.int64(3), np.float64(0.0)), (0, np.int64(4), np.float64(0.0)), (0, np.int64(5), np.float64(0.0)), (0, np.int64(6), np.float64(0.0)), (0, np.int64(7), np.float64(0.0)), (0, np.int64(8), np.float64(0.0)), (0, np.int64(9), np.float64(0.0)), (0, np.int64(10), np.float64(0.0)), (0, np.int64(11), np.float64(0.0)), (0, np.int64(12), np.float64(0.0)), (0, np.int64(13), np.float64(0.0)), (1, np.int64(2), np.float64(0.0)), (1, np.int64(3), np.float64(0.0)), (1, np.int64(4), np.float64(0.0)), (1, np.int64(5), np.float64(0.0)), (1, np.int64(6), np.float64(0.0)), (1, np.int64(7), np.float64(0.0)), (1, np.int64(8), np.float64(0.0)), (1, np.int64(9), np.float64(0.0)), (1, np.int64(10), np.float64(0.0)), (1, np.int64(11), np.float64(0.0)), (1, np.int64(12), np.float64(0.0)), (1, np.int64(13), np.float64(0.0)), (2, np.int64(3), np.float64(0.0)), (2, np.int64(4), np.float64(0.0))

In [ ]:
# Using the results
vecs_alpha = Water_wfn.Ca().np 
vecs_beta = Water_wfn.Cb().np
#print(vecs)
vals_alpha = Water_wfn.epsilon_a().np
vals_beta = Water_wfn.epsilon_b().np
Counter(vals_alpha) == Counter(vals_beta)
#for i,tau in enumerate(taus):
#    mu_upper[i] = mu1
#    
#    func = particle_number_shifter(tau,vals,vecs,x,10)
#    mu1,fx0 = secant_method(mu0,mu1,func,criterion=1e-10)
#    
#    mu0s[i] = mu0
#    mu1s[i] = mu1
    
#    mus[i] = mu1
#    Nes[i] = particle_number_function(mu1,tau,vals,vecs,x)
    #print(Nes[i])
#    mu0 = mu1-(dtau+(.1*dtau*i))

### Searching over $\tau$ range and returning $\mu$ for each $\tau$ to conserve number of particles 

In [ ]:
def tau_search(taus,mu0,mu1,vals,vecs,x,Ne,criterion=1e-10):
    '''
    INPUT:
        taus: vector, len=n
            the tau value to find the temperature dependent mu for
        mu0: scalar
            a guess at a value below the chemical potential for taus[0]
        mu1: scalar
            a guess for a value above the chemical potential for taus[0]
        vals: vector, len=k
            The eigenvalues of the eigenvectors that the mus should be computed for
        vecs: matrix, size=(Nx,k)
            The eigenvectors of the system that mu should be found for for each tau.
            Nx is the number of grid points.
            k is the number of states included in the calculation.
        x: vector, len=Nx
            The grid that the eigenvectors were computed on.
        Ne: scalar
            The fixed number of electrons in the system
        criterion: scalar
            The convergence criterion that the secant method should look to obtain
    OUTPUT
        mus: vector, len=n
            the chemical potnetial at each temp tau to conserve the number of particles in the system
    '''
    dtau = taus[1]-taus[0]
    mus = np.empty(len(taus))

    for i,tau in enumerate(taus):
        
        func = particle_number_shifter(tau,vals,vecs,x,Ne)
        mu1,fx0 = secant_method(mu0,mu1,func,criterion=criterion)
    
        mus[i] = mu1
    
        mu0 = mu1-(dtau+(.1*dtau*i))
        
    return mus

In [ ]:
ntaus = 101
taus = np.linspace(1,20,ntaus)
Ne = 10
mus_alpha = tau_search(taus,1,5,vals_alpha,vecs_alpha,x,Ne,criterion=1e-10)
mus_beta = tau_search(taus,1,5,vals_beta,vecs_beta,x,Ne,criterion=1e-10)

#vecs.shape

In [ ]:
dens = np.empty((Nx,ntaus))
fss = np.empty((nk,ntaus))
for i,mu in enumerate(mus_alpha):
    fs_alpha = fermi_weighting(vals_alpha,mu,taus[i])
for i, mu in enumerate(mus_beta):
    fs_beta = fermi_weighting(vals_beta,mu,taus[i])
    #print(f"fermi weight number {i} is : {fs[i]}")
    #print(f"The fermi weight for eigenvector {i+1} is :eigenvector = {vecs[1,i]} fermi weight = {fs[i]}")
    #fss[:,i] = fs
    #dens[:,i] = dens_function(vecs, x, fs, FWs="yes")
print(fs_alpha)
print(fs_beta)
Counter(fs_alpha) == Counter(fs_beta)
#plt.plot(fss)
#plt.xlabel('')
#plt.ylabel('')
#plt.title("Fermi Occupations")
#plt.show()

In [ ]:
#---------------> Plot
fig, ax = plt.subplots(ncols=1, dpi=100) #sharex=True, sharey=True, dpi=200)
#---------------> Data
ax.plot(x,dens[:,1])#, linewidth=3,)
#---------------> Formatting

ax.set_xlabel('postion ($b_0$)')#, fontsize=20)
ax.set_ylabel('density (e/$b_0^3$)')#, fontsize=20)
fig.suptitle("Thermal Water Density")#, fontsize=20)
fig.tight_layout()
#-------> Borders ## Have to run cell twice for this to work no idea why -- AO
#plt.rcParams["axes.edgecolor"] = "black"
#plt.rcParams["axes.linewidth"] = 2.0
#plt.xticks(fontsize=15)
#plt.yticks(fontsize=15)
#ax.xaxis.set_tick_params(width=2)
#ax.yaxis.set_tick_params(width=2)

In [ ]:
#---------------> Plot
fig, ax = plt.subplots(ncols=3, dpi=200) #sharex=True, sharey=True, dpi=200)
#---------------> Data
ax[0].plot(x,dens[:,0])
ax[1].plot(x,dens[:,50])
ax[2].plot(x,dens[:,100])
#---------------> Formatting
ax[0].set_box_aspect(1)
ax[1].set_box_aspect(1)
ax[2].set_box_aspect(1)

ax[0].set_xlabel('postion ($b_0$)')
ax[1].set_xlabel('postion ($b_0$)')
ax[2].set_xlabel('postion ($b_0$)')
ax[0].set_ylabel('density (e/$b_0^3$)')
ax[1].set_ylabel('density (e/$b_0^3$)')
ax[2].set_ylabel('density (e/$b_0^3$)')
ax[0].title.set_text('at $\\tau = 1.0 $')
ax[1].title.set_text('at $\\tau = 10.5 $')
ax[2].title.set_text('at $\\tau = 20.0 $')

fig.tight_layout()
#-------> Borders ## Have to run cell twice for this to work no idea why -- AO
#plt.rcParams["axes.edgecolor"] = "black"
#plt.rcParams["axes.linewidth"] = 1.00


In [ ]:
fig, ax = plt.subplots(1,1, dpi=100)
#---------------> Data
line = ax.plot(x,dens[:,0])[0]
#---------------> Formatting
ax.set_xlabel('postion ($b_0$)')
ax.set_ylabel('Density ((e/$b_0)^3$)')
#---------------> Animation
def update(frame):
    line.set_data(x,dens[:,frame+1])
    return line
ani = animation.FuncAnimation(fig=fig, func=update, frames=ntaus-1, interval=30)
writergif = animation.PillowWriter(fps=30)
ani.save('thermal_occ_water.gif',writer=writergif)